<a href="https://colab.research.google.com/github/Ivan412/training/blob/main/6_python_advaced/4_reports%20and%20automation/task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Продвинутый Python. Часть 4: Отчеты и автоматизация. Базовые отчеты.

---


#**Задание: геральдические символы Москвы**

Сгенерируйте PDF документ из списка флагов и гербов районов Москвы:

https://video.ittensive.com/python-advanced/data-102743-2019-11-13.utf.csv

На каждой странице документа выведите название геральдического символа (Name), его описание (Description) и его изображение (Picture).

Для показа изображений используйте адрес

https://op.mos.ru/MEDIA/showFile?id=XXX

где XXX - это значение поля Picture в наборе данных. Например:

https://op.mos.ru/MEDIA/showFile?id=8466da35-6801-41a9-a71e-04b60408accb

In [ ]:
%%capture
#скачаем шрифт
!wget http://video.ittensive.com/python-advanced/Trebuchet.ttf

In [ ]:
%%capture
#обновим модули
!pip install --upgrade Pillow reportlab

In [ ]:
#перезапустим рантайм
exit()

In [ ]:
import pandas as pd
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont
from reportlab.pdfgen import canvas
from reportlab.lib import pagesizes
from reportlab.lib.utils import ImageReader
import requests
import time

In [ ]:
def add_page(PDF, name, desc, pic):
  '''
  добавляем типовую страницу с гербом
  '''
  url = 'http://op.mos.ru/MEDIA/showFile?id='

  #название геральдического символа
  y = 780
  tu = 24
  PDF.setFont("Trebuchet", tu)
  name = split_line(name, tu) 
  for line in name:
    PDF.drawString(line_offset(line, tu), y, line)
    y -= 20
  
  #герб
  #с колаба нет доступа на op.mos.ru, поэтому идем через прокси
  proxies = {'http': 'http://5.35.93.34:8080',
             'https': 'https://95.174.102.131:53281'}
  y -= 300
  while True:
    try:
      time.sleep(1)
      resp = requests.get(url+pic, stream=True, proxies=proxies)
      PDF.drawImage(ImageReader(resp.raw), 100, y)
      break
    except:
      print('Proxy\image error occurred')

  #описание
  y -= 100
  tu = 18
  PDF.setFont("Trebuchet", tu)
  desc = split_line(desc, tu)
  for line in desc:
    PDF.drawString(line_offset(line, tu), y, line)
    y -= 15
  PDF.showPage()
  return PDF

def split_line(line, tu):
  '''
  формируем строки нужной длины в зависимости от размера шрифта
  поддерживаются размеры 24 и 18
  '''
  #максимум символов в строке в от кегля, после вычитания отступов
  max_chars = {24: 40, 18: 48}[tu]
  result = []
  words = line.strip().split(' ')
  line = []
  for word in words:
    #в пустой лист добавляем слово
    #добавим новое слово, если оно не приведет к превышению порога символов
    if (len(line) == 0) or (line_len(line) + len(word) < max_chars):
      line.append(word)
    #если слово не влезло, каст листа к строке идобавление слова в новый лист
    else:
      result.append(' '.join(line))
      line = []
      line.append(word)
  #последнее слово влезло в лист(сработал if на последней итерации выше)
  #добавляем в результат и его
  if line:
    result.append(' '.join(line))
  return result


def line_len(line):
  '''
  считаем сумму длин слов в листе и пробелов между ними
  '''
  result = sum(map(len, line)) + len(line) - 1
  return result


def line_offset(line, tu):
  '''
  считаем левый отступ в координатах для строки 
  произвольной длины 24го или 18 размера
  '''
  #ширина строки в символах от кегля
  max_chars = {24: 50, 18: 65}[tu]
  #ширина символа от кегля
  mult = {24: 12.5, 18: 9}[tu]
  #из макс длины в символах вычтем длину строки поделим на 
  #2(левый отступ) и переведем в координаты
  offset = round(((max_chars - len(line)) / 2) * mult) 

  return offset

In [ ]:
#загрузим данные
data  = pd.read_csv('https://video.ittensive.com/python-advanced/data-102743-2019-11-13.utf.csv', delimiter=';', index_col='ID')

In [ ]:
#создадим титульный лист
pdfmetrics.registerFont(TTFont('Trebuchet', 'Trebuchet.ttf'))
PDF = canvas.Canvas("symbols.pdf", pagesize = pagesizes.A4)
PDF.setFont("Trebuchet", 48)
PDF.drawString(120, 550, "Геральдические")
PDF.drawString(190, 490, "символы")
PDF.drawString(200, 430, "Москвы")
PDF.showPage()


In [ ]:
#создадим страницы с геральдикой
for ind, row in data.iterrows():
  name, desc, pic = row['Name'], row['Description'], row['Picture']
  print(f'Создаю страницу... {name}')
  PDF = add_page(PDF, name, desc, pic)
print('Страницы созданы')

Создаю страницу... Герб муниципального округа Новокосино в городе Москве
Создаю страницу... Флаг муниципального округа Новокосино в городе Москве
Создаю страницу... Герб муниципального округа Нагатино-Садовники в городе Москве
Proxy\image error occurred
Proxy\image error occurred
Proxy\image error occurred
Proxy\image error occurred
Proxy\image error occurred
Proxy\image error occurred
Proxy\image error occurred
Proxy\image error occurred
Proxy\image error occurred
Proxy\image error occurred
Proxy\image error occurred
Proxy\image error occurred
Proxy\image error occurred
Создаю страницу... Флаг муниципального округа Нагатино-Садовники в городе Москве
Proxy\image error occurred
Proxy\image error occurred
Proxy\image error occurred
Proxy\image error occurred
Proxy\image error occurred
Proxy\image error occurred
Proxy\image error occurred
Proxy\image error occurred
Proxy\image error occurred
Proxy\image error occurred
Proxy\image error occurred
Proxy\image error occurred
Proxy\image error

In [ ]:
PDF.save()